In [ ]:
import pandas as pd
import numpy as np

# ===================== 1. CONFIGURACIÓN =====================
# Ruta de tu archivo (ajústala según tu caso)
ruta_csv = "/content/drive/MyDrive/SIS330/Proyecto_dietas/dataset_peso_semana.csv"

# Cargar dataset original
# Ejemplo de columnas esperadas: persona, peso_inicial_lb, peso_final_lb, semanas_totales
data = pd.read_csv(ruta_csv)
print("✅ Dataset original cargado:")
print(data.head(), "\n")

# ===================== 2. CONVERSIÓN A KILOGRAMOS =====================
def libras_a_kg(libras):
    return libras * 0.453592

data["peso_inicial_kg"] = data["peso_inicial_lb"].apply(libras_a_kg)
data["peso_final_kg"] = data["peso_final_lb"].apply(libras_a_kg)

print("📏 Conversión a kilogramos realizada.")
print(data[["persona", "peso_inicial_kg", "peso_final_kg", "semanas_totales"]].head(), "\n")

# ===================== 3. FUNCIÓN PARA GENERAR PESOS INTERMEDIOS =====================
def generar_pesos_semana(peso_ini, peso_fin, semanas, curva="exponencial"):
    """
    Genera una lista de pesos por semana, simulando pérdida progresiva.
    """
    if curva == "lineal":
        return np.linspace(peso_ini, peso_fin, semanas)
    elif curva == "exponencial":
        # Pérdida más rápida al inicio, luego más lenta
        x = np.linspace(0, 1, semanas)
        decay = 1 - np.exp(-3 * x)
        pesos = peso_ini - (peso_ini - peso_fin) * decay
        return pesos
    else:
        raise ValueError("Tipo de curva no soportado")

# ===================== 4. GENERAR DATASET EXPANDIDO =====================
registros = []

for _, row in data.iterrows():
    pesos = generar_pesos_semana(row["peso_inicial_kg"], row["peso_final_kg"], int(row["semanas_totales"]))
    for semana, peso in enumerate(pesos, start=1):
        registros.append({
            "persona": row["persona"],
            "semana": semana,
            "peso_kg": round(peso, 2)
        })

df_expandido = pd.DataFrame(registros)
print("📈 Ejemplo de datos generados:")
print(df_expandido.head(10))

# ===================== 5. GUARDAR NUEVO DATASET =====================
ruta_salida = "/content/drive/MyDrive/SIS330/Proyecto_dietas/dataset_peso_expandido.csv"
df_expandido.to_csv(ruta_salida, index=False)

print(f"\n✅ Dataset expandido guardado en: {ruta_salida}")
